In [1]:
import cv2
import os
import argparse

import numpy as np

In [2]:
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", type=str,
	default=os.path.sep.join(["images", "adrian.jpg"]),
	help="path to input image that we'll apply GrabCut to")
ap.add_argument("-c", "--iter", type=int, default=10,
	help="# of GrabCut iterations (larger value => slower runtime)")
args = vars(ap.parse_args())

usage: ipykernel_launcher.py [-h] [-i IMAGE] [-c ITER]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/carwyn/.local/share/jupyter/runtime/kernel-4b31f259-12b7-4105-8461-ec0d7b943aee.json


SystemExit: 2

/home/carwyn/anaconda3/envs/hypertrophy/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# For ipynb, simulate inputs
image = "../../Media/Images/image1"

In [ ]:
image = cv2.imread(args["image"])
mask = np.zeros(image.shape[:2], dtype="uint8")

rect = (151, 43, 236, 368)